In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
file_test = 'test_spam.csv'

data_test = pd.read_csv(file_test)

In [4]:
data_test.head()

,text
0,j jim whitehead ejw cse ucsc edu writes j you ...
1,original message from bitbitch magnesium net p...
2,java for managers vince durasoft who just taug...
3,there is a youtuber name saiman says
4,underpriced issue with high return on equity t...


In [5]:
data_test.shape

(4070, 1)

In [6]:
data_test.describe()

,text
count,4070
unique,4070
top,j jim whitehead ejw cse ucsc edu writes j you ...
freq,1


In [21]:
data_test.iloc[0, 0]

'j jim whitehead ejw cse ucsc edu writes j you open sourced the new components you developed for this j project so the next person who comes along won t have to j reimplement them right no need all those components already exist either in the java class libraries or from the various java jar collections most of the classes i used came from the jakarta project and apachexml but if it s any consolation my threading of them all together into a newswire server is gpl and available on sourceforge gary lawrence murphy garym teledyn com teledynamics communications inc business advantage through community software url computers are useless they can only give you answers pablo picasso'

In [24]:
file_train = 'train_spam.csv'

data_train = pd.read_csv(file_train)

In [25]:
data_train

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...
...,...,...
16273,spam,if you are interested in binary options tradin...
16274,spam,dirty pictureblyk on aircel thanks you for bei...
16275,ham,or you could do this g on mon 1635465 sep 1635...
16276,ham,insta reels par 80 गंद bhara pada hai 👀 kuch b...


In [28]:
data_train.describe()

,text_type,text
count,16278,16278
unique,2,16267
top,ham,SPAM ALERT 🚔 User: Username: @DillyBubbl...
freq,11469,7


In [32]:
print(f"Число нормальных сообщений: {sum(data_train['text_type'] == 'ham')}")
print(f"Число спама: {sum(data_train['text_type'] == 'spam')}")

Число нормальных сообщений: 11469
Число спама: 4809


In [27]:
data_train.iloc[0, 1]

'make sure alex knows his birthday is over in fifteen minutes as far as youre concerned'